# Watertight ShapeNetSem

## INPUTS

```
{input_acronym_dir}
├── 1Shelves_12a64182bbaee7a12b2444829a3507de_0.00914554366969263.h5
├── 1Shelves_160684937ae737ec5057ad0f363d6ddd_0.009562610447288044.h5
├── 1Shelves_1e3df0ab57e8ca8587f357007f9e75d1_0.011099225885734912.h5
├── 1Shelves_2b9d60c74bc0d18ad8eae9bce48bbeed_0.00614208274225087.h5

{input_shapenetsem_dir}
├── 1004f30be305f33d28a1548e344f0e2e.mtl
├── 1004f30be305f33d28a1548e344f0e2e.obj
├── 100f39dce7690f59efb94709f30ce0d2.mtl
├── 100f39dce7690f59efb94709f30ce0d2.obj
```

## OUTPUTS

```
{output_shapenetsem_dir}
└── meshes
    ├── Mug
    │   └── 10f6e09036350e92b3f21f1137c3c347.obj
    └── Table
        └── 99cf659ae2fe4b87b72437fd995483b.obj
```

## NOTES

* 24980 files in ShapeNetSem, half are mtl and half are obj, so total of 12490
* 8836 files in grasps from acronym
* each acronym file should have a corresponding shapenetsem file (8836 x 2 outputs for obj and mtl)
* each acronym file has a filename path to mesh file that looks like meshes/Mug/10f6e09036350e92b3f21f1137c3c347.obj
* the shapenetsem files have a name but they don't match exactly
* there can be more than one mesh per category (eg. multiple under the TV directory)
* there can also be more than one acronym file per mesh (eg. multiple grasps for the same mesh )
  eg. "data/grasps/TV_dfbce5e6cca00c1448627a76b6268107_0.0038627305095302734.h5"
      "data/grasps/TV_dfbce5e6cca00c1448627a76b6268107_0.003957748840105706.h5"

SHOULD USE MANIFOLD (https://github.com/hjwdzh/Manifold), BUILD, AND COPY THIS SCRIPT TO THE MANIFOLD ROOT DIRECTORY

In [1]:
import subprocess
import os
from tqdm import tqdm
from typing import DefaultDict

In [2]:
# INPUT PARAMS
input_acronym_dir = "/juno/u/tylerlum/github_repos/acronym/data/grasps/"
input_shapenetsem_dir = "/juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/"
temp_watertight_obj_filename = "temp.watertight.obj"
output_watertight_shapenetsem_dir = "WatertightShapeNetSem/models"

In [3]:
print("=" * 100)
print(f"PARAMS")
print("=" * 100)
print(f"input_acronym_dir: {input_acronym_dir}")
print(f"input_shapenetsem_dir: {input_shapenetsem_dir}")
print(f"temp_watertight_obj_filename: {temp_watertight_obj_filename}")
print(f"output_watertight_shapenetsem_dir: {output_watertight_shapenetsem_dir}")
print()

PARAMS
input_acronym_dir: /juno/u/tylerlum/github_repos/acronym/data/grasps/
input_shapenetsem_dir: /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/
temp_watertight_obj_filename: temp.watertight.obj
output_watertight_shapenetsem_dir: WatertightShapeNetSem/models



In [4]:
# Check inputs
if not os.path.exists(input_acronym_dir):
    print(f"input_acronym_dir: {input_acronym_dir} does not exist. Exiting.")
    exit()

if not os.path.exists(input_shapenetsem_dir):
    print(f"input_shapenetsem_dir: {input_shapenetsem_dir} does not exist. Exiting.")
    exit()

if os.path.exists(temp_watertight_obj_filename):
    print(
        f"temp_watertight_obj_filename: {temp_watertight_obj_filename} already exists. Removing it and continuing."
    )
    subprocess.run(f"rm {temp_watertight_obj_filename}", shell=True, check=True)

if os.path.exists(output_watertight_shapenetsem_dir):
    print(
        f"output_watertight_shapenetsem_dir: {output_watertight_shapenetsem_dir} already exists. Exiting."
    )
    exit()

In [5]:
# Get acronym obj hashes
print("=" * 100)
print(f"ACRONYM OBJ HASHES")
print("=" * 100)
acronym_filenames = os.listdir(input_acronym_dir)
print(f"Found {len(acronym_filenames)} files in {input_acronym_dir}")
print(f"First 10 acronym_filenames: {acronym_filenames[:10]}")
acronym_obj_hashes = [filename.split("_")[1] for filename in acronym_filenames]
print(f"First 10 acronym_obj_hashes: {acronym_obj_hashes[:10]}")

ACRONYM OBJ HASHES
Found 8836 files in /juno/u/tylerlum/github_repos/acronym/data/grasps/
First 10 acronym_filenames: ['Monitor_ab8cd6dd65cb0ca9770f31b5d5715b63_0.006682695306575539.h5', 'Armoire_28bf43e37ce672cbe3f7a74e12a274ef_0.00013782695626756404.h5', 'Bench_552ab19eab47725cb2a965e75be701c_0.0017012528560007953.h5', 'Sideboard_bd89eb4a7407f07e54d8afaf6caac97c_0.005737483591372497.h5', 'TV_525b558141c5a1287b105c57586eb4ca_0.00196817753413085.h5', 'Horse_18277a2b26cce4945932aa8e4205e707_0.003072872918754287.h5', 'Laptop_164b84d13b3d4650e096f6db40afe7bf_0.0043268075537907045.h5', 'TV_83be8773a6f0abe9eb65e4b964e1175e_0.002272723453505538.h5', 'CeilingLamp_1facaa0eb35496fb69783ac218a8a2bf_0.0026773719342714597.h5', 'TrashBin_8689c0f170e62baa3243977ac99cb30_0.0021906069043480404.h5']
First 10 acronym_obj_hashes: ['ab8cd6dd65cb0ca9770f31b5d5715b63', '28bf43e37ce672cbe3f7a74e12a274ef', '552ab19eab47725cb2a965e75be701c', 'bd89eb4a7407f07e54d8afaf6caac97c', '525b558141c5a1287b105c57586eb4ca

In [6]:
# Print information about acronym obj hashes (some may be off)
len_to_hashes = DefaultDict(list)
for obj_hash in acronym_obj_hashes:
    len_to_hashes[len(obj_hash)].append(obj_hash)
len_to_freq = {length: len(hashes) for length, hashes in len_to_hashes.items()}
print(f"len_to_freq: {len_to_freq}")
print("Example of each:")
for length, hashes in len_to_hashes.items():
    print(f"length: {length}, hash: {hashes[0]}")
acronym_obj_hashes = set(acronym_obj_hashes)
print()

len_to_freq: {32: 7777, 31: 914, 30: 105, 6: 26, 29: 14}
Example of each:
length: 32, hash: ab8cd6dd65cb0ca9770f31b5d5715b63
length: 31, hash: 552ab19eab47725cb2a965e75be701c
length: 30, hash: 5b8a2074b476c79f7ff6ace05b36a5
length: 6, hash: room20
length: 29, hash: 16bb10ad445ecfdaa476d4febccee



In [7]:
# Get all filenames
print("=" * 100)
print(f"SHAPENET FILENAMES")
print("=" * 100)
shapenet_filenames = os.listdir(input_shapenetsem_dir)
print(f"Found {len(shapenet_filenames)} files in {input_shapenetsem_dir}")
print(f"First 10 shapenet_filenames: {shapenet_filenames[:10]}")
shapenet_filenames.sort()

SHAPENET FILENAMES
Found 24980 files in /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/
First 10 shapenet_filenames: ['3cdd2855b459d71199bdf8d5a28e79e9.obj', '46187b0273450a8826a62c9cd09c9b29.mtl', 'f0fce14f8394ae7c118ddfdb81cc6068.obj', '802134affda120ae95741efb2dbe17e5.mtl', '7cadb6cf9d282125916bc112690303c9.obj', '66cf0a36c4bcef8e1f0d77760f3a597.mtl', '23851ee504b8653db4bb73dbde2fe319.mtl', 'dee9e56b9946037a226eabe1cca3e850.obj', 'aa9b45add3d76150f54e1b6f41fdd78a.obj', '600fd2598fd5761eb9099be21c212692.mtl']


In [8]:
obj_filenames = [
    filename for filename in shapenet_filenames if filename.endswith(".obj")
]
mtl_filenames = [
    filename for filename in shapenet_filenames if filename.endswith(".mtl")
]
assert len(obj_filenames) == len(mtl_filenames)

In [9]:
# Sanity check
for obj_filename, mtl_filename in zip(obj_filenames, mtl_filenames):
    obj_filename_without_ext, obj_ext = os.path.splitext(obj_filename)
    mtl_filename_without_ext, mtl_ext = os.path.splitext(mtl_filename)
    assert obj_filename_without_ext == mtl_filename_without_ext
    assert obj_ext == ".obj"
    assert mtl_ext == ".mtl"

In [10]:
# Filter out non-acronym obj hashes
obj_filenames = [
    obj_filename
    for obj_filename in obj_filenames
    if os.path.splitext(obj_filename)[0] in acronym_obj_hashes
]
mtl_filenames = [
    mtl_filename
    for mtl_filename in mtl_filenames
    if os.path.splitext(mtl_filename)[0] in acronym_obj_hashes
]
print(
    f"Filtered down to {len(obj_filenames)} obj_filenames and {len(mtl_filenames)} mtl_filenames"
)
print()

Filtered down to 7897 obj_filenames and 7897 mtl_filenames



In [11]:
# Make output dir
print(f"Making output dir: {output_watertight_shapenetsem_dir}")
os.makedirs(output_watertight_shapenetsem_dir)
print(f"Done making output dir: {output_watertight_shapenetsem_dir}")

Making output dir: WatertightShapeNetSem/models
Done making output dir: WatertightShapeNetSem/models


In [12]:
num_failed = 0
for obj_filename, mtl_filename in (
    pbar := tqdm(zip(obj_filenames, mtl_filenames), total=len(obj_filenames))
):
    pbar.set_description(f"num_failed: {num_failed}")
    try:
        obj_filename_without_ext, obj_ext = os.path.splitext(obj_filename)
        mtl_filename_without_ext, mtl_ext = os.path.splitext(mtl_filename)
        assert obj_filename_without_ext == mtl_filename_without_ext
        assert obj_ext == ".obj"
        assert mtl_ext == ".mtl"

        input_obj_filepath = os.path.join(input_shapenetsem_dir, obj_filename)
        watertight_mesh_command = (
            f"./build/manifold {input_obj_filepath} {temp_watertight_obj_filename} -s"
        )
        print(f"watertight_mesh_command: {watertight_mesh_command}")
        subprocess.run(watertight_mesh_command, shell=True, check=True)

        output_obj_filepath = os.path.join(
            output_watertight_shapenetsem_dir, obj_filename
        )
        simplify_command = f"./build/simplify -i {temp_watertight_obj_filename} -o {output_obj_filepath} -m -r 0.02"
        print(f"simplify_command: {simplify_command}")
        subprocess.run(simplify_command, shell=True, check=True)

        rm_command = f"rm {temp_watertight_obj_filename}"
        subprocess.run(rm_command, shell=True, check=True)

        print()
    except subprocess.CalledProcessError as e:
        num_failed += 1
        print("=" * 100)
        print(
            f"Error: {e} when processing {obj_filename} and {mtl_filename}. Skipping it."
        )
        print("=" * 100)
        print()

num_failed: 10:   0%|    | 0/7897 [00:00<?, ?it/s]
/bin/sh: 1: ./build/manifold: not found
num_failed: 14:   0%|    | 0/7897 [00:00<?, ?it/s]/bin/sh: 1: 

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/1004f30be305f33d28a1548e344f0e2e.obj temp.watertight.obj -s
Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/1004f30be305f33d28a1548e344f0e2e.obj temp.watertight.obj -s' returned non-zero exit status 127. when processing 1004f30be305f33d28a1548e344f0e2e.obj and 1004f30be305f33d28a1548e344f0e2e.mtl. Skipping it.

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/100f39dce7690f59efb94709f30ce0d2.obj temp.watertight.obj -s
Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/100f39dce7690f59efb94709f30ce0d2.obj temp.watertight.obj -s' returned non-zero exit status 127. when processing 100f39dce7690f59efb94709f30ce0d2.obj and 100f39dce7690f59efb94709f30ce0d2.mtl. Skipping it.

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/

./build/manifold: not found
/bin/sh: 1: ./build/manifold: not found
num_failed: 17:   0%|    | 0/7897 [00:00<?, ?it/s]/bin/sh: 1: ./build/manifold: not found
num_failed: 18:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 19:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 20:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 21:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 22:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 23:   0%| | 18/7897 [00:00<00:45, 173.

Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/1049af17ad48aaeb6d41c42f7ade8c8.obj temp.watertight.obj -s' returned non-zero exit status 127. when processing 1049af17ad48aaeb6d41c42f7ade8c8.obj and 1049af17ad48aaeb6d41c42f7ade8c8.mtl. Skipping it.

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/10507ae95f984daccd8f3fe9ca2145e1.obj temp.watertight.obj -s
Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/10507ae95f984daccd8f3fe9ca2145e1.obj temp.watertight.obj -s' returned non-zero exit status 127. when processing 10507ae95f984daccd8f3fe9ca2145e1.obj and 10507ae95f984daccd8f3fe9ca2145e1.mtl. Skipping it.

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/1061c1b4af7fd99777f4e9e0a7e4c2.obj temp.watertight.obj -s
Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/data/S

/bin/sh: 1: ./build/manifold: not found
num_failed: 24:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 25:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 26:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 27:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 28:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 29:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 31:   0%| | 18/7897 [00:00<00:45, 173../build/manifold: not found
num_failed: 32:   0%| | 18/7897 [00:00<00:45, 173.
num_failed: 33:   0%| | 18/7897 [00:00<00:45, 173.
/bin/sh: 1: ./build/manifold: not found
num_failed: 34:   0%| | 18/7897 [00:00<00:45, 173./bin/sh: 1: ./build/manifold: not found
num_failed: 35:   0%| | 18/7897 [00:00<00:45, 173.

...

/bin/sh: 1: ./build/manifold: not found
num_failed: 134:   2%| | 129/7897 [00:00<00:42, 18/bin/sh: 1: ./build/manifold: not found
/bin/sh: 1: ./build/manifold: not found
num_failed: 135:   2%| | 129/7897 [00:00<00:42, 18/bin/sh: 1: ./build/manifold: not found
num_failed: 137:   2%| | 129/7897 [00:00<00:42, 18./build/manifold: not found
num_failed: 138:   2%| | 129/7897 [00:00<00:42, 18./build/manifold: not found
num_failed: 139:   2%| | 129/7897 [00:00<00:42, 18./build/manifold: not found
/bin/sh: 1: ./build/manifold: not found
num_failed: 140:   2%| | 129/7897 [00:00<00:42, 18/bin/sh: 1: ./build/manifold: not found
num_failed: 141:   2%| | 129/7897 [00:00<00:42, 18/bin/sh: 1: ./build/manifold: not found
num_failed: 143:   2%| | 129/7897 [00:00<00:42, 18./build/manifold: not found
/bin/sh: 1: ./build/manifold: not found
num_failed: 144:   2%| | 129/7897 [00:00<00:42, 18

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/1389e932a2c776d83c143af07c12991a.obj temp.watertight.obj -s
Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/1389e932a2c776d83c143af07c12991a.obj temp.watertight.obj -s' returned non-zero exit status 127. when processing 1389e932a2c776d83c143af07c12991a.obj and 1389e932a2c776d83c143af07c12991a.mtl. Skipping it.

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/13961f08fe5b4c549ef60e9225cbea3a.obj temp.watertight.obj -s
Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/13961f08fe5b4c549ef60e9225cbea3a.obj temp.watertight.obj -s' returned non-zero exit status 127. when processing 13961f08fe5b4c549ef60e9225cbea3a.obj and 13961f08fe5b4c549ef60e9225cbea3a.mtl. Skipping it.

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/

/bin/sh: 1: ./build/manifold: not found
num_failed: 145:   2%| | 129/7897 [00:00<00:42, 18/bin/sh: 1: ./build/manifold: not found
num_failed: 147:   2%| | 129/7897 [00:00<00:42, 18./build/manifold: not found
/bin/sh: 1: ./build/manifold: not found
num_failed: 148:   2%| | 148/7897 [00:00<00:42, 18/bin/sh: 1: ./build/manifold: not found
num_failed: 149:   2%| | 149/7897 [00:00<00:44, 17
/bin/sh: 1: ./build/manifold: not found


Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/13c480eb5d2a1f4b71c690692c393680.obj temp.watertight.obj -s' returned non-zero exit status 127. when processing 13c480eb5d2a1f4b71c690692c393680.obj and 13c480eb5d2a1f4b71c690692c393680.mtl. Skipping it.

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/13c8bf6b41e555b78821ad3452f1c8c9.obj temp.watertight.obj -s
Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/13c8bf6b41e555b78821ad3452f1c8c9.obj temp.watertight.obj -s' returned non-zero exit status 127. when processing 13c8bf6b41e555b78821ad3452f1c8c9.obj and 13c8bf6b41e555b78821ad3452f1c8c9.mtl. Skipping it.

watertight_mesh_command: ./build/manifold /juno/u/tylerlum/github_repos/acronym/data/ShapeNetSem/models/13dbeeacdabce3b694658a0201ba0367.obj temp.watertight.obj -s
Error: Command './build/manifold /juno/u/tylerlum/github_repos/acronym/d


KeyboardInterrupt

